## Pipeline for realML.kernel.RFMPreconditionedGaussianKRR and realML.kernel.RFMPreconditionedPolynomialKRR

converted from the example pipeline for the r_26 radon prediction dataset: expects the `r_26` directory to be in the working directory

In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import cross_val_score, ShuffleSplit, cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
import warnings
warnings.filterwarnings('ignore')
from realML.kernel import RFMPreconditionedGaussianKRR, RFMPreconditionedPolynomialKRR

In [3]:
# load training data
rootDir = './r_26'
dataDir = os.path.join(rootDir,'data')
assert os.path.exists(dataDir)
rawDir = os.path.join(dataDir,'raw_data')
assert os.path.exists(rawDir)

trainData = pd.read_csv(os.path.join(dataDir, 'trainData.csv'), index_col=0)
X_train = pd.read_csv(os.path.join(rawDir, 'radon_train.csv'), index_col=0)
y_train = pd.read_csv(os.path.join(dataDir, 'trainTargets.csv'), index_col=0)

In [4]:
# manually select the following two features ['county', 'floor']
# selected by reading the problem and data descriptions
X_train = X_train[['county', 'floor']]

# both county and floor are categorical variables, requires one-hot encoding
X_train = pd.get_dummies(X_train, columns = ['county', 'floor'])
#print(X_train.head())

In [5]:
# using the primitives directly (kind of useless, 
# since the new API means we can't use cross validate to check the model,
# or grid search to find the parameters,
# but informative)
gaussKRR = RFMPreconditionedGaussianKRR(lparam = 1.0, sigma=1/np.sqrt(2*.1))
gaussKRR.set_training_data(inputs=[X_train.as_matrix()], outputs=[y_train.as_matrix()])
gaussKRR.fit()
ypred = gaussKRR.produce(inputs=[X_train.as_matrix()])[0]
np.linalg.norm(ypred - y_train.as_matrix())/np.sqrt(len(ypred))

0.71476255733869487

In [6]:
# create wrapper sklearn estimator classes so can use cross_validate
class gaussKRREstimator(BaseEstimator, RegressorMixin):
    def __init__(self, lparam=1, sigma=1):
        self.fastEstimator = RFMPreconditionedGaussianKRR(lparam=1, sigma=1)
    
    def fit(self, X, y):
        self.fastEstimator.set_training_data(inputs=[X.as_matrix()], outputs=[y.as_matrix()])
        self.fastEstimator.fit()
    
    def predict(self, X):
        return self.fastEstimator.produce(inputs=[X.as_matrix()])[0]
    
    def get_params(self, deep=True):
        return {"lparam": self.fastEstimator.lparam, 
                "sigma": self.fastEstimator.sigma}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self.fastEstimator, parameter, value)
        return self
    
class polyKRREstimator(BaseEstimator, RegressorMixin):
    def __init__(self, lparam=1, offset = 0, sf = 1):
        self.fastEstimator = RFMPreconditionedPolynomialKRR(lparam = 1, offset = 0, sf = 1)
    
    def fit(self, X, y):
        self.fastEstimator.set_training_data(inputs=[X.as_matrix()], outputs=[y.as_matrix()])
        self.fastEstimator.fit()
    
    def predict(self, X):
        return self.fastEstimator.produce(inputs=[X.as_matrix()])[0]
    
    def get_params(self, deep=True):
        return {"lparam": self.fastEstimator.lparam, 
                "offset": self.fastEstimator.offset,
                "sf": self.fastEstimator.sf}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self.fastEstimator, parameter, value)
        return self

In [7]:
gaussKRR = gaussKRREstimator(lparam=1.0, sigma=1/np.sqrt(2*(0.1)))
gaussKRR.fit(X_train, y_train)
predy = gaussKRR.predict(X_train)

polyKRR = polyKRREstimator(lparam=1.0, offset=1.0, sf=1)
polyKRR.fit(X_train, y_train)
predy = gaussKRR.predict(X_train)

1268.8624827 1319.75073404


In [8]:
# do cross validation to select best gaussKRR fit
RMSE = lambda yT, yP: np.sqrt(mean_squared_error(yT, yP))

print('trying model: Gaussian kernel ridge...')
cv = ShuffleSplit(n_splits=10, test_size=0.25, random_state=0)
kr = GridSearchCV(
    gaussKRREstimator(lparam=1.0, sigma=0.1), 
    cv=cv,
    param_grid={"lparam": [1e0, 0.1, 1e-2, 1e-3],
                "sigma": np.logspace(-2, 3, 7)},
    scoring=make_scorer(RMSE, greater_is_better=False)
)
kr.fit(X_train, y_train)
score = kr.best_score_ # that score is negative MSE scores. The thing is that GridSearchCV, by convention, always tries to maximize its score so loss functions like MSE have to be negated.
score = score*-1
print('model performance on 10-fold CV (mean rmse)', score)
print(kr.best_estimator_)

trying model: Gaussian kernel ridge...
model performance on 10-fold CV (mean rmse) 0.761491708167
gaussKRREstimator(lparam=0.01, sigma=21.544346900318846)


In [15]:
# do cross validation to select best polyKRR fit
RMSE = lambda yT, yP: np.sqrt(mean_squared_error(yT, yP))

print('trying model: Polynomial kernel ridge...')
cv = ShuffleSplit(n_splits=10, test_size=0.25, random_state=0)
kr = GridSearchCV(
    polyKRREstimator(lparam=1.0, offset=0.1, sf=1.0), 
    cv=cv,
    param_grid={"lparam": [1e0, 0.1, 1e-2, 1e-3],
                "offset": np.logspace(-2, 3, 7),
                "sf": np.logspace(-2,3,7)},
    scoring=make_scorer(RMSE, greater_is_better=False)
)
kr.fit(X_train, y_train)
score = kr.best_score_ # that score is negative MSE scores. The thing is that GridSearchCV, by convention, always tries to maximize its score so loss functions like MSE have to be negated.
score = score*-1
print('model performance on 10-fold CV (mean rmse)', score)
print(kr.best_estimator_)

trying model: Polynomial kernel ridge...
model performance on 10-fold CV (mean rmse) 0.760978877808
polyKRREstimator(lparam=0.001, offset=0.068129206905796158, sf=0.01)
